In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.metrics import classification_report, f1_score, accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import random
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv("EMB_all_final.csv")
data = data.drop(['Unnamed: 0'], 1)
data.iloc[44410,-2] = -1
data

,userName,date,content,score,if_apple,if_useful,crush,data,keyboard,time/life,IP,system upgrad,connection,other-tech,money,UI_UX,event,mechanism,ad
0,Jocelyn Sy,2022-10-27 18:10:20,fun,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,crystal may,2022-10-27 12:17:07,aw matter game close progress spent coin gem l...,1,0,1.0,0.0,-1.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,Coreter,2022-10-27 07:07:17,cool,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,Jordan Christie,2022-10-26 15:11:40,use love game recent new go sync progress acco...,2,0,1.0,0.0,0.0,0.0,0.0,0.0,0,-1,-1,0.0,0.0,0.0,0.0,0.0
4,Heather Nicole,2022-10-26 03:42:00,love game dont know shut time clear make sure ...,4,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163653,jfsajk,2017-12-02 06:57:59,great,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
163654,ty lol ty,2022-09-26 07:38:26,fun,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
163655,pogdw,2022-09-20 21:43:42,fun,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0
163656,arajuju,2022-08-03 12:04:10,love,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

polarity = [round(sid_obj.polarity_scores(i)['compound'], 2) for i in data['content']]
data['senti_score'] = polarity

In [4]:
data.columns

Index(['userName', 'date', 'content', 'score', 'if_apple', 'if_useful',
       'crush', 'data', 'keyboard', 'time/life', 'IP', 'system upgrad',
       'connection', 'other-tech', 'money', 'UI_UX', 'event', 'mechanism',
       'ad', 'senti_score'],
      dtype='object')

In [5]:
names = ['crush', 'data', 'keyboard', 'time/life', 'IP', 'system upgrad',
       'connection', 'other-tech', 'money', 'UI_UX', 'event', 'mechanism',
       'ad']
for i in names:
    data[i] = data[i].astype("category")

In [6]:
# whole, apple, google
data = data
#data = data[data['if_apple'] == 1] # apple
#data = data[data['if_apple'] != 1] # google

In [7]:
y = data['score']

In [8]:
X_dummy = pd.get_dummies(data,columns = names)
X_dummy = X_dummy.drop(['userName','date','content','score'],1)
X_dummy = X_dummy.drop(['senti_score'],1) # 不要senti
X_dummy

,if_apple,if_useful,crush_-1.0,crush_0.0,data_-1.0,data_0.0,data_1.0,keyboard_-1.0,keyboard_0.0,keyboard_1.0,...,UI_UX_1.0,event_-1.0,event_0.0,event_1.0,mechanism_-1.0,mechanism_0.0,mechanism_1.0,ad_-1.0,ad_0.0,ad_1.0
0,0,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
1,0,1.0,0,1,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
2,0,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
3,0,1.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
4,0,1.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163653,1,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
163654,1,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
163655,1,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
163656,1,0.0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0


In [9]:
X_train_dummy, X_test_dummy, y_train, y_test = train_test_split(X_dummy,y,test_size = 0.2,random_state=0)

In [127]:
clf = SGDClassifier()
clf = clf.fit(X_train_dummy,y_train)
y_pred = clf.predict(X_test_dummy)
accuracy_score(y_pred,y_test)

0.6940608578760845

In [128]:
f1_score(y_pred,y_test,average = "weighted")

0.7860942209432313

In [129]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.28      0.35      0.31      2386
           2       0.09      0.02      0.03      1165
           3       0.00      0.00      0.00      2025
           4       1.00      0.00      0.00      4622
           5       0.74      0.97      0.84     22534

    accuracy                           0.69     32732
   macro avg       0.42      0.27      0.24     32732
weighted avg       0.67      0.69      0.60     32732



/Users/wuyue/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [130]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = SGDClassifier()
clf2 = RandomForestClassifier()
clf = VotingClassifier(estimators = [('sgd',clf1),('rf',clf2)])

In [131]:
clf = clf.fit(X_train_dummy,y_train)
y_pred = clf.predict(X_test_dummy)
accuracy_score(y_pred,y_test)

0.7002627398264695

In [132]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.34      0.35      0.34      2386
           2       0.14      0.00      0.01      1165
           3       0.21      0.01      0.02      2025
           4       0.32      0.03      0.05      4622
           5       0.74      0.97      0.84     22534

    accuracy                           0.70     32732
   macro avg       0.35      0.27      0.25     32732
weighted avg       0.60      0.70      0.61     32732



In [133]:
clf = RandomForestClassifier()
clf = clf.fit(X_train_dummy,y_train)
y_pred = clf.predict(X_test_dummy)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.40      0.22      0.29      2386
           2       0.14      0.00      0.01      1165
           3       0.14      0.01      0.01      2025
           4       0.26      0.05      0.08      4622
           5       0.73      0.98      0.84     22534

    accuracy                           0.70     32732
   macro avg       0.33      0.25      0.24     32732
weighted avg       0.58      0.70      0.61     32732



In [109]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.36      0.28      0.31      2386
           2       0.13      0.00      0.01      1165
           3       0.19      0.04      0.06      2025
           4       0.25      0.01      0.02      4622
           5       0.73      0.98      0.84     22534

    accuracy                           0.70     32732
   macro avg       0.33      0.26      0.25     32732
weighted avg       0.58      0.70      0.61     32732



In [57]:
clf = RandomForestClassifier()
clf = clf.fit(X_train_dummy,y_train)
y_pred = clf.predict(X_test_dummy)
accuracy_score(y_pred,y_test)

0.6978491995600635

In [58]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.41      0.33      0.37      2386
           2       0.13      0.04      0.06      1165
           3       0.19      0.07      0.10      2025
           4       0.25      0.07      0.11      4622
           5       0.76      0.96      0.85     22534

    accuracy                           0.70     32732
   macro avg       0.35      0.29      0.30     32732
weighted avg       0.60      0.70      0.63     32732



In [29]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.40      0.33      0.36      2386
           2       0.12      0.04      0.06      1165
           3       0.19      0.07      0.10      2025
           4       0.26      0.07      0.11      4622
           5       0.76      0.96      0.85     22534

    accuracy                           0.70     32732
   macro avg       0.35      0.29      0.30     32732
weighted avg       0.60      0.70      0.63     32732



In [38]:
X_train, X_test, y_train, y_test = train_test_split(data['content'],y,test_size = 0.2,shuffle = True,random_state = 0)

tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 100)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
df_X_train = pd.DataFrame(X_train_vectors_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
df_X_test = pd.DataFrame(X_test_vectors_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())

In [39]:
clf = RandomForestClassifier()
clf = clf.fit(df_X_train,y_train)
y_pred = clf.predict(df_X_test)
accuracy_score(y_pred,y_test)

0.7153244531345472

In [40]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.44      0.45      0.44      2386
           2       0.17      0.02      0.04      1165
           3       0.25      0.11      0.15      2025
           4       0.35      0.14      0.20      4622
           5       0.78      0.95      0.86     22534

    accuracy                           0.72     32732
   macro avg       0.40      0.34      0.34     32732
weighted avg       0.64      0.72      0.66     32732



In [49]:
data['score'].value_counts()

5    112415
4     22827
1     11941
3     10622
2      5853
Name: score, dtype: int64